In [1]:
import pandas as pd
import numpy as np
import os
import yfinance as yf
from datetime import timedelta

In [181]:
SPX_Prices = yf.download('SPY', start='2002-6-30', end='2023-11-01', interval = "1d")
SPX_Prices

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2002-01-02,115.110001,115.750000,113.809998,115.529999,76.324287,18651900
2002-01-03,115.650002,116.949997,115.540001,116.839996,77.189743,15743000
2002-01-04,117.169998,117.980003,116.550003,117.620003,77.705093,20140700
2002-01-07,117.699997,117.989998,116.559998,116.790001,77.156662,13106500
2002-01-08,116.790001,117.059998,115.970001,116.519997,76.978333,12683700
...,...,...,...,...,...,...
2023-12-22,473.859985,475.380005,471.700012,473.649994,473.649994,67126600
2023-12-26,474.070007,476.579987,473.989990,475.649994,475.649994,55387000
2023-12-27,475.440002,476.660004,474.890015,476.510010,476.510010,68000300


In [149]:
VIX = yf.download('^VIX', start='2002-01-01', end='2024-01-01', interval = "1d")
VIX

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2002-01-02,23.780001,24.200001,22.709999,22.709999,22.709999,0
2002-01-03,22.219999,22.430000,21.330000,21.340000,21.340000,0
2002-01-04,20.969999,21.530001,20.400000,20.450001,20.450001,0
2002-01-07,21.410000,22.150000,21.350000,21.940001,21.940001,0
2002-01-08,21.629999,22.290001,21.280001,21.830000,21.830000,0
...,...,...,...,...,...,...
2023-12-22,13.720000,13.960000,13.000000,13.030000,13.030000,0
2023-12-26,13.770000,13.800000,12.960000,12.990000,12.990000,0
2023-12-27,13.020000,13.040000,12.370000,12.430000,12.430000,0


In [173]:
def garman_klass_daily_variance(data):
    """
    Calculate daily Garman-Klass variance for given price data.
    """
    log_hl = np.log(data['High'] / data['Low'])
    log_co = np.log(data['Close'] / data['Open'])
    daily_variance = 0.5 * log_hl**2 - (2 * np.log(2) - 1) * log_co**2
    return daily_variance

Date
2020-06-30    0.000137
2020-07-01    0.000038
2020-07-02    0.000073
2020-07-06    0.000021
2020-07-07    0.000077
                ...   
2023-10-25    0.000027
2023-10-26    0.000061
2023-10-27    0.000058
2023-10-30    0.000049
2023-10-31    0.000045
Length: 841, dtype: float64


Date
2020-06-30    0.000137
2020-07-01    0.000038
2020-07-02    0.000073
2020-07-06    0.000021
2020-07-07    0.000077
                ...   
2023-10-25    0.000027
2023-10-26    0.000061
2023-10-27    0.000058
2023-10-30    0.000049
2023-10-31    0.000045
Name: Daily_Volatility, Length: 841, dtype: float64

In [178]:
def rolling_volatility(data, rolling_window):
    """
    Calculate annualized Garman-Klass volatility over a given period
    """
    Daily_Volatility = garman_klass_daily_variance(data)
    Rolling_Vol = np.sqrt((Daily_Volatility.rolling(5).mean())*252)
    return Rolling_Vol

In [179]:
weekly_vol = rolling_volatility(SPX_Prices, 5)
monthly_vol = rolling_volatility(SPX_Prices, 21)
quartely_vol = rolling_volatility(SPX_Prices, 63)

In [188]:
volatilities = pd.DataFrame(
    data = {"weekly_vol" : weekly_vol,
            "monthly_vol" : monthly_vol,
            "quartely_vol" : quartely_vol
           },
    index = (SPX_Prices.index))

volatilities = volatilities.dropna()
volatilities

,weekly_vol,monthly_vol,quartely_vol
Date,,,
2002-01-08,0.129403,0.129403,0.129403
2002-01-09,0.138090,0.138090,0.138090
2002-01-10,0.138751,0.138751,0.138751
2002-01-11,0.134745,0.134745,0.134745
2002-01-14,0.129609,0.129609,0.129609
...,...,...,...
2023-12-22,0.095185,0.095185,0.095185
2023-12-26,0.092716,0.092716,0.092716
2023-12-27,0.092868,0.092868,0.092868
